## Домашнее задание 3. Деревья решений на игрушечном примере и датасете UCI Adult. Решение

Начнём с загрузки необходимых библиотек.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

### Часть 1. Игрушечный датасет «Пойдёт — не пойдёт?»

Цель — разобраться в работе деревьев решений на игрушечном примере. Хотя одно дерево решений не даёт выдающихся результатов, другие мощные алгоритмы (градиентный бустинг, случайный лес) основаны на той же идее.

Рассмотрим игрушечный пример бинарной классификации — Персона A решает, пойдёт ли она на второе свидание с Персоной B. Это зависит от внешности, красноречия, употребления алкоголя и суммы потраченных денег.

#### Создание датасета

In [ ]:
# Создаём DataFrame с dummy-переменными
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Некоторые значения признаков есть в train, но отсутствуют в test и наоборот.
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [ ]:
features = ['Looks', 'Alcoholic_beverage','Eloquence','Money_spent']

#### Обучающие данные

In [ ]:
df_train = {}
df_train['Looks'] = ['handsome', 'handsome', 'handsome', 'repulsive',
                         'repulsive', 'repulsive', 'handsome'] 
df_train['Alcoholic_beverage'] = ['yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes']
df_train['Eloquence'] = ['high', 'low', 'average', 'average', 'low',
                                   'high', 'average']
df_train['Money_spent'] = ['lots', 'little', 'lots', 'little', 'lots',
                                  'lots', 'lots']
df_train['Will_go'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

#### Тестовые данные

In [ ]:
df_test = {}
df_test['Looks'] = ['handsome', 'handsome', 'repulsive'] 
df_test['Alcoholic_beverage'] = ['no', 'yes', 'yes']
df_test['Eloquence'] = ['average', 'high', 'average']
df_test['Money_spent'] = ['lots', 'little', 'lots']
df_test = create_df(df_test, features)
df_test

In [ ]:
# Некоторые значения признаков есть в train, но отсутствуют в test и наоборот.
y = df_train['Will_go']
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

In [ ]:
df_test

#### Нарисуйте дерево решений для этого датасета.

1\. Чему равна энтропия $S_0$ начальной системы?

**Ответ:** $S_0 = -\frac{3}{7}\log_2{\frac{3}{7}}-\frac{4}{7}\log_2{\frac{4}{7}} = 0.985$.

2\. Разобьём данные по признаку "Looks_handsome". Чему равны энтропии $S_1$, $S_2$ и информационный выигрыш (IG)?

**Ответ:** $S_1 = -\frac{1}{4}\log_2{\frac{1}{4}}-\frac{3}{4}\log_2{\frac{3}{4}} = 0.811$, $S_2 = -\frac{2}{3}\log_2{\frac{2}{3}}-\frac{1}{3}\log_2{\frac{1}{3}} = 0.918$, $IG = S_0-\frac{4}{7}S_1-\frac{3}{7}S_2 = 0.128$.

#### Обучим дерево решений с помощью sklearn.

In [ ]:
dt = DecisionTreeClassifier(criterion='entropy', random_state=17)
dt.fit(df_train, y);

#### Визуализация дерева:

In [ ]:
plot_tree(dt, feature_names=df_train.columns, filled=True,
         class_names=["Won't go", "Will go"]);

### Часть 2. Функции для вычисления энтропии и информационного выигрыша.

Разминочный пример: 9 синих шаров и 11 жёлтых. Метка **1** — синий, **0** — иначе.

In [ ]:
balls = [1 for i in range(9)] + [0 for i in range(11)]

<img src='https://habrastorage.org/webt/mu/vl/mt/muvlmtd2njeqf18trbldenpqvnm.png'>

Разделим шары на две группы:

<img src='https://habrastorage.org/webt/bd/aq/5w/bdaq5wi3c4feezaexponvin8wmo.png'>

In [ ]:
# две группы
balls_left  = [1 for i in range(8)] + [0 for i in range(5)] # 8 синих и 5 жёлтых
balls_right = [1 for i in range(1)] + [0 for i in range(6)] # 1 синий и 6 жёлтых

#### Реализация функции для вычисления энтропии Шеннона

In [ ]:
from math import log
    
def entropy(a_list):
    lst = list(a_list)
    size = len(lst) 
    entropy = 0
    set_elements = len(set(lst))
    if set_elements in [0, 1]:
        return 0
    for i in set(lst):
        occ = lst.count(i)
        entropy -= occ/size * log (occ/size,2)
    return entropy

Тесты

In [ ]:
print(entropy(balls)) # 9 синих и 11 жёлтых
print(entropy(balls_left)) # 8 синих и 5 жёлтых
print(entropy(balls_right)) # 1 синий и 6 жёлтых
print(entropy([1,2,3,4,5,6])) # энтропия честного шестигранного кубика

3\. Чему равна энтропия состояния, заданного списком **balls_left**?

**Ответ:** 0.961

4\. Чему равна энтропия честного кубика?

**Ответ:** 2.585

In [ ]:
# вычисление информационного выигрыша
def information_gain(root, left, right):
    '''root — начальные данные, left и right — два разбиения'''
        
    return entropy(root) - 1.0 * len(left) / len(root) * entropy(left) \
                         - 1.0 * len(right) / len(root) * entropy(right) 

In [ ]:
print(information_gain(balls, balls_left, balls_right))

5\. Чему равен информационный выигрыш при разбиении на **balls_left** и **balls_right**?

**Ответ:** 0.161

In [ ]:
def information_gains(X, y):
    '''Возвращает информационный выигрыш при разбиении по каждому признаку'''
    out = []
    for i in X.columns:
        out.append(information_gain(y, y[X[i] == 0], y[X[i] == 1]))
    return out

#### Опционально: реализация построения дерева

In [ ]:
information_gains(df_train, y)

In [ ]:
def btree(X, y, feature_names):
    clf = information_gains(X, y)
    best_feat_id = clf.index(max(clf))
    best_feature = feature_names[best_feat_id]
    print (f'Лучший признак для разбиения: {best_feature}')
    
    x_left = X[X.iloc[:, best_feat_id] == 0]
    x_right = X[X.iloc[:, best_feat_id] == 1]
    print (f'Объектов: {len(x_left)} (слева) и {len(x_right)} (справа)')
    
    y_left = y[X.iloc[:, best_feat_id] == 0]
    y_right = y[X.iloc[:, best_feat_id] == 1]
    entropy_left = entropy(y_left)
    entropy_right = entropy(y_right)
    print (f'Энтропия: {entropy_left} (слева) и {entropy_right} (справа)')
    print('_' * 30 + '\n')
    if entropy_left != 0:
        print(f'Разбиваем левую группу ({len(x_left)} объектов):')
        btree(x_left, y_left, feature_names)
    if entropy_right != 0:
        print(f'Разбиваем правую группу ({len(x_right)} объектов):')
        btree(x_right, y_right, feature_names)

In [ ]:
btree (df_train, y, df_train.columns)

Визуализация далека от идеала, но легко сопоставляется с нормальной визуализацией дерева (через sklearn) выше.

### Часть 3. Датасет "Adult"

#### Описание датасета:

[Датасет](http://archive.ics.uci.edu/ml/machine-learning-databases/adult) UCI Adult: классифицировать людей по демографическим данным — зарабатывают ли они более \$50 000 в год или нет.

Описание признаков:

- **Age** — непрерывный признак
- **Workclass** — непрерывный признак
- **fnlwgt** — итоговый вес объекта, непрерывный признак
- **Education** — категориальный признак
- **Education_Num** — число лет обучения, непрерывный признак
- **Martial_Status** — категориальный признак
- **Occupation** — категориальный признак
- **Relationship** — категориальный признак
- **Race** — категориальный признак
- **Sex** — категориальный признак
- **Capital_Gain** — непрерывный признак
- **Capital_Loss** — непрерывный признак
- **Hours_per_week** — непрерывный признак
- **Country** — категориальный признак

**Target** — уровень дохода, категориальный (бинарный) признак.

#### Чтение обучающих и тестовых данных

In [ ]:
data_train = pd.read_csv('../data/adult_train.csv')

In [ ]:
data_train.tail()

In [ ]:
data_test = pd.read_csv('../data/adult_test.csv')

In [ ]:
data_test.tail()

In [ ]:
# удаляем строки с некорректными метками в тестовом датасете
data_test = data_test[(data_test['Target'] == ' >50K.') | (data_test['Target']==' <=50K.')]

# кодируем целевую переменную как целое число
data_train.loc[data_train['Target']==' <=50K', 'Target'] = 0
data_train.loc[data_train['Target']==' >50K', 'Target'] = 1

data_test.loc[data_test['Target']==' <=50K.', 'Target'] = 0
data_test.loc[data_test['Target']==' >50K.', 'Target'] = 1

data_train['Target'] = data_train['Target'].astype(int)
data_test['Target'] = data_test['Target'].astype(int)

#### Первичный анализ данных

In [ ]:
data_test.describe(include='all').T

In [ ]:
data_train['Target'].value_counts()

In [ ]:
fig = plt.figure(figsize=(25, 15))
cols = 5
rows = int(np.ceil(float(data_train.shape[1]) / cols))
for i, column in enumerate(data_train.columns):
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    if data_train.dtypes[column] == object:
        data_train[column].value_counts().plot(kind="bar", axes=ax)
    else:
        data_train[column].hist(axes=ax)
        plt.xticks(rotation="vertical")
plt.subplots_adjust(hspace=0.7, wspace=0.2)

#### Проверка типов данных

In [ ]:
data_train.dtypes

In [ ]:
data_test.dtypes

В тестовых данных возраст имеет тип **object**. Исправим это.

In [ ]:
data_test['Age'] = data_test['Age'].astype(int)

Приведём все **float**-признаки к типу **int**.

In [ ]:
data_test['fnlwgt'] = data_test['fnlwgt'].astype(int)
data_test['Education_Num'] = data_test['Education_Num'].astype(int)
data_test['Capital_Gain'] = data_test['Capital_Gain'].astype(int)
data_test['Capital_Loss'] = data_test['Capital_Loss'].astype(int)
data_test['Hours_per_week'] = data_test['Hours_per_week'].astype(int)

#### Заполним пропущенные данные: для непрерывных — медианой, для категориальных — модой.

In [ ]:
# видим пропущенные значения
data_train.info()

In [ ]:
# выбираем категориальные и непрерывные признаки

categorical_columns = [c for c in data_train.columns 
                       if data_train[c].dtype.name == 'object']
numerical_columns = [c for c in data_train.columns 
                     if data_train[c].dtype.name != 'object']

print('categorical_columns:', categorical_columns)
print('numerical_columns:', numerical_columns)

In [ ]:
# заполняем пропуски

for c in categorical_columns:
    data_train[c].fillna(data_train[c].mode()[0], inplace=True)
    data_test[c].fillna(data_train[c].mode()[0], inplace=True)
    
for c in numerical_columns:
    data_train[c].fillna(data_train[c].median(), inplace=True)
    data_test[c].fillna(data_train[c].median(), inplace=True)

In [ ]:
# пропусков больше нет
data_train.info()

Закодируем категориальные признаки с помощью `pd.get_dummies`.

In [ ]:
data_train = pd.concat([data_train[numerical_columns],
    pd.get_dummies(data_train[categorical_columns])], axis=1)

data_test = pd.concat([data_test[numerical_columns],
    pd.get_dummies(data_test[categorical_columns])], axis=1)

In [ ]:
set(data_train.columns) - set(data_test.columns)

In [ ]:
data_train.shape, data_test.shape

#### В тестовых данных нет Голландии. Создадим новый признак с нулевыми значениями.

In [ ]:
data_test['Country_ Holand-Netherlands'] = 0
data_test = data_test[data_train.columns]

In [ ]:
set(data_train.columns) - set(data_test.columns)

In [ ]:
data_train.head(2)

In [ ]:
data_test.head(2)

In [ ]:
X_train = data_train.drop(['Target'], axis=1)
y_train = data_train['Target']

X_test = data_test.drop(['Target'], axis=1)
y_test = data_test['Target']

### 3.1 Дерево решений без настройки параметров

Обучим дерево решений с максимальной глубиной 3 и **random_state = 17**.

In [ ]:
tree = DecisionTreeClassifier(max_depth=3, random_state=17)
tree.fit(X_train, y_train)

Предсказание на тестовых данных.

In [ ]:
tree_predictions = tree.predict(X_test) 

In [ ]:
accuracy_score(y_test, tree_predictions)

6\. Accuracy на тестовой выборке для дерева с max_depth=3: **0.845**

### 3.2 Дерево решений с настройкой параметров

Найдём оптимальную максимальную глубину с помощью 5-fold кросс-валидации.

In [ ]:
%%time
tree_params = {'max_depth': range(2, 11)}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=17),
                                 tree_params, cv=5)                  

locally_best_tree.fit(X_train, y_train)

In [ ]:
print("Best params:", locally_best_tree.best_params_)
print("Best cross validaton score", locally_best_tree.best_score_)

Обучим дерево с max_depth=9 и вычислим accuracy на тестовой выборке.

In [ ]:
tuned_tree = DecisionTreeClassifier(max_depth=9, random_state=17)
tuned_tree.fit(X_train, y_train)
tuned_tree_predictions = tuned_tree.predict(X_test)
accuracy_score(y_test, tuned_tree_predictions)

7\. Accuracy на тестовой выборке для дерева с max_depth=9: **0.848**

### 3.3 (Опционально) Случайный лес без настройки параметров

Забежим вперёд и попробуем случайный лес.

Обучим случайный лес со 100 деревьями и **random_state = 17**.

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=17)
rf.fit(X_train, y_train)

Кросс-валидация.

In [ ]:
%%time
cv_scores = cross_val_score(rf, X_train, y_train, cv=3)

In [ ]:
cv_scores, cv_scores.mean()

Предсказания на тестовых данных.

In [ ]:
forest_predictions = rf.predict(X_test) 

In [ ]:
accuracy_score(y_test, forest_predictions)

### 3.4 (Опционально) Случайный лес с настройкой параметров

Обучим случайный лес из 10 деревьев. Настроим max_depth и max_features через **GridSearchCV**.

In [ ]:
forest_params = {'max_depth': range(10, 16),
                 'max_features': range(5, 105, 20)}

locally_best_forest = GridSearchCV(
    RandomForestClassifier(n_estimators=10, random_state=17,
                           n_jobs=4),
    forest_params, cv=3, verbose=1, n_jobs=4)

locally_best_forest.fit(X_train, y_train)

In [ ]:
print("Best params:", locally_best_forest.best_params_)
print("Best cross validaton score", locally_best_forest.best_score_)

Предсказания на тестовых данных.

In [ ]:
tuned_forest_predictions = locally_best_forest.predict(X_test) 
accuracy_score(y_test, tuned_forest_predictions)

С настройкой параметров лес из 10 деревьев показывает результат лучше, чем лес из 100 деревьев с параметрами по умолчанию.